## Обработка файлов курьеров и подбор минимальных признаков

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 1000)     

### Скачиваем данные и обрезаем для удобства анализа

In [181]:
# ПОМЕНЯТЬ ДАТУ
date_split = '2024-02-01'
file_path = '09.csv'

In [ ]:
df = pd.read_csv(f'data/{file_path}')

date_columns = [
    'date_status', 'first_date_order', 'last_date_order', 'date_supply', 'date_supply_untill', 'date_collected', 'date_delivery_start', 'date_delivered', 'pool_date_start']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    # Округление до минут (обнуляем секунды и микросекунды)
    df[col] = df[col].dt.floor('min')

df['date'] = df['date_delivery_start'].dt.date
df['id_courier'] = df['id_courier'].fillna(0)
df['id_courier'] = df['id_courier'].astype(int)
df = df[df['id_courier'] > 0]
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.sort_values('date_delivery_start',inplace=True)


### Фильтрация данных и очистка от пропусков 

In [183]:
df = df[df['date_delivered'] > df['date_delivery_start']]
df = df[df['date_status'] > df['first_date_order']]

df['courier_type']=df['courier_type'].fillna('Тип курьера неизвестен')
df.dropna(subset=['id_courier'], inplace=True)
df.drop(columns=['id_courier.1'], inplace=True)

nan_counts = df.isna().sum()
print(nan_counts)

# Фильтрация по источникам
vv_source = [21, 1, 15, 78, 79, 67, 80, 71, 33, 2, 5, 3, 4, 39, 66, 58, 63]
df = df[df['id_source'].isin(vv_source)]

df=df[(~df['date_delivery_start'].isna())&(~df['date_delivered'].isna())]
df['total_order_weight'] = df['total_order_weight'].fillna(0)

df.drop(columns=['distance','pool_date_start'],inplace=True)
nan_counts = df.isna().sum()
print(nan_counts)


id_pool                     0
id_courier                  0
courier_type                0
id_job                      0
pool_date_start         14038
sort_order                  0
id_general                  0
distance               648712
status_str                  0
date_status                 0
id_general.1                0
total_order_weight          5
total_orders                0
first_date_order            0
last_date_order             0
date_supply                 0
date_supply_untill          0
date_collected              0
date_delivery_start         0
date_delivered              0
id_service                  0
id_source                   0
latitude_order              0
longitude_order             0
N                           0
shirota_tt                  0
dolgota_tt                  0
gen_distance           110024
date                        0
dtype: int64
id_pool                     0
id_courier                  0
courier_type                0
id_job                     

### Обработка по ТТ

In [185]:
# 1. Создаём 15-минутные интервалы
min_date = df['date_delivery_start'].min().floor('15min')
max_date = df['date_delivery_start'].max().ceil('15min')
date_range = pd.date_range(start=min_date, end=max_date, freq='15min')

# 2. Округляем время доставки до 15-минутного интервала
df['date_group'] = df['date_delivery_start'].dt.floor('15min')
tts = df['N'].unique()
# 3. Создаём каркас данных: все комбинации курьеров и временных интервалов
tt = df['N'].unique()
grid = pd.MultiIndex.from_product([tts, date_range], names=['N', 'date_group'])
tt_df = pd.DataFrame(index=grid).reset_index()

# 4. Определяем смены (разрыв между заказами > 2.5 часа)
df = df.sort_values(['N', 'date_delivery_start'])
# 5. Привязываем смены к 15-минутным интервалам
tt_df = tt_df.merge(df[['N', 'date_group', 'id_courier']],
                              on=['N', 'date_group'], how='left')
tt_df

,N,date_group,id_courier
0,8145.0,2024-09-01 00:00:00,1.799963e+09
1,8145.0,2024-09-01 00:00:00,1.799963e+09
2,8145.0,2024-09-01 00:15:00,1.799963e+09
3,8145.0,2024-09-01 00:30:00,1.799879e+09
4,8145.0,2024-09-01 00:45:00,NaN
...,...,...,...
6024854,8448.0,2024-10-01 20:00:00,NaN
6024855,8448.0,2024-10-01 20:15:00,NaN
6024856,8448.0,2024-10-01 20:30:00,NaN
6024857,8448.0,2024-10-01 20:45:00,NaN


In [ ]:
active_couriers_df = tt_df.groupby(['N', 'date_group'])['id_courier'] \
                         .nunique() \
                         .reset_index() \
                         .rename(columns={'id_courier': 'active_couriers_count'})

,N,date_group,active_couriers_count
0,191.0,2024-09-01 00:00:00,0
1,191.0,2024-09-01 00:15:00,0
2,191.0,2024-09-01 00:30:00,0
3,191.0,2024-09-01 00:45:00,0
4,191.0,2024-09-01 01:00:00,0
...,...,...,...
1518075,8502.0,2024-10-01 20:00:00,0
1518076,8502.0,2024-10-01 20:15:00,0
1518077,8502.0,2024-10-01 20:30:00,0
1518078,8502.0,2024-10-01 20:45:00,0


In [ ]:
TT_df = active_couriers_df.copy()
null_tt = TT_df[TT_df['active_couriers_count'] == 0]
null_tt = null_tt['N'].value_counts().reset_index()
null_tt.columns = ['N', 'zero_count']

TT_df = TT_df.merge(null_tt, on='N', how='left')
TT_list_df = pd.read_excel('ТТ.xlsx')

TT_df = TT_df.merge(TT_list_df[['N', 'name_TT', 'is_active', 'Статус', 'tt_format_rasp', 'region_tt']], on='N', how='left')


TT_df['тип_ТТ'] = np.where(
    TT_df['name_TT'].str.contains(r'\d+М_', regex=True),
    'Магазин',
    np.where(TT_df['name_TT'].str.contains(r'\d+ДС_', regex=True),
             'ДС',
             None)
)

In [189]:
TT_df = TT_df[TT_df['tt_format_rasp'] == 700]
TT_df = TT_df[['N','date_group','active_couriers_count']]

In [190]:
TT_df.to_csv(f'data_2/{file_path}', index=False)


In [191]:
import os

def combine_files(folder_path, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            
            if os.path.isfile(file_path):  # Ensure it's a file, not a directory
                with open(file_path, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read())
                    outfile.write('\n')  # Add a newline between files
    print(f"All files combined into {output_file}")

# Example usage
folder_path = "data_2"  # Change this to your folder path
output_file = "processed_tt.csv"  # Change this to your desired output file name
combine_files(folder_path, output_file)


All files combined into processed_tt.csv
